<a href="https://colab.research.google.com/github/salbifaza/digital-skola-homework-week-8-spark/blob/main/HW_Spark_Salbi_Faza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework - Spark
Nama: Salbi Faza Rinaldi

Bootcamp Data Engineer Batch 15


## Instruksi Tugas

*  Buat program menggunakan SparkSQL untuk mencari berapa Amount minimum yang dikeluarkan untuk setiap CustomerID.
*  Tentukan Schema dari DataFrame menggunakan StructType seperti pada contoh code min-temprature-dataframe.py
*  Column pada dataset retail-data-full.csv antara lain: InvoiceNo, StockCode, Description, Quantity, InvoiceData, Amount, CustomerID, Country
*  File menggunakan separated “;”
*  Simpan pada variable results, kemudian print
*  Silahkan gunakan code program min-temprature-dataframe.py pada pertemuan sebelumnya sebagai referensi untuk menyelesaikan tugas ini
*  Kumpulkan tugas dalam bentuk pdf atau attach link google collab



## Clone Git Repositories

In [ ]:
!git clone https://github.com/salbifaza/digital-skola-homework-week-8-spark

## Install Requirement Module

In [1]:
!pip install -q findspark

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=0277547a6367b0387c6771ad737a54f331ac47bba6d0204b54ef34edca263db6
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


## Import & Initiate Moduel


In [3]:
# Import & initate Findspark
import findspark
findspark.init()

In [15]:
# Import Spark Session
from pyspark.sql import SparkSession

# Import Spark Data Types
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, IntegerType, FloatType

# Import Spark Function
from pyspark.sql import functions as func

## Spark Code


In [5]:
# Create Spark Session
spark = SparkSession.builder.appName('CustomerMoV').getOrCreate()

In [35]:
# Define Schema DataFrame
schema = StructType([ \
                     StructField('InvoiceNo', IntegerType(), True), \
                     StructField('StockCode', StringType(), True), \
                     StructField('Description', StringType(), True), \
                     StructField('Quantity', IntegerType(), True), \
                     StructField('InvoiceDate', TimestampType(), True), \
                     StructField('Amount', FloatType(), True), \
                     StructField('CustomerID', FloatType(), True), \
                     StructField('Country', StringType(), True)

])

In [36]:
# Read File
csv_file = "/content/digital-skola-homework-week-8-spark/Data session 23 Spark II - retail-data-full.csv"
results = spark.read.option("delimiter", ";").schema(schema).csv(csv_file)

In [37]:
# Print File
results.show()

+---------+---------+--------------------+--------+-------------------+------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|Amount|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|  2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|  3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|  2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|  3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|  3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|  7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|2010

In [40]:
# Get Customer & Minimum Amount using DataFrame
CustomerAmount = results.select('CustomerID','Amount')
CustomerAmount.groupBy("CustomerID").min('Amount').sort('CustomerID').show(100)

In [38]:
# Get Customer & Minimum Amount using SparkSQL
results.createOrReplaceTempView('results')
spark.sql('select CustomerID, min(Amount) from results group by CustomerID order by CustomerID').show(100)

## Additional

In [73]:
# Hourly Transaction
spark.sql("select date_format(InvoiceDate, 'y-MM-dd HH:00') as time, count(distinct InvoiceNo) as trx from results group by 1 order by 1").show(100)

+----------------+---+
|            time|trx|
+----------------+---+
|2010-12-01 08:00|  6|
|2010-12-01 09:00| 16|
|2010-12-01 10:00| 11|
|2010-12-01 11:00| 13|
|2010-12-01 12:00| 22|
|2010-12-01 13:00| 12|
|2010-12-01 14:00| 19|
|2010-12-01 15:00| 15|
|2010-12-01 16:00| 17|
|2010-12-01 17:00|  6|
+----------------+---+



In [78]:
# Top 10 Stock by Qty Sold
spark.sql("select StockCode, Description, sum(quantity) as total_qty from results group by 1,2 order by 3 desc").show(10)

+---------+--------------------+---------+
|StockCode|         Description|total_qty|
+---------+--------------------+---------+
|    17021|NAMASTE SWAGAT IN...|      600|
|   85099B|JUMBO BAG RED RET...|      556|
|   84029E|RED WOOLLY HOTTIE...|      551|
|    21232|STRAWBERRY CERAMI...|      549|
|    21137|BLACK RECORD COVE...|      540|
|    21731|RED TOADSTOOL LED...|      483|
|    22466|FAIRY TALE COTTAG...|      462|
|   85123A|WHITE HANGING HEA...|      454|
|    20668|DISCO BALL CHRIST...|      344|
|    22095|LADS ONLY TISSUE BOX|      338|
+---------+--------------------+---------+
only showing top 10 rows

